# Utils

> Utils for indexing and searching.

In [ ]:
# | default_exp index.utils


In [ ]:
# | export

from dreamai_ray.imports import *
from dreamai_ray.utils import *
from dreamai_ray.mapper import *


In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# | export


def read_ems(
    df,
    ems_col="embedding",
    ems_key="embedding",
    task_folder=None,
    task_id=gen_random_string(16),
):
    ems = df[ems_col]
    if path_or_str(ems):
        ems_path, _ = handle_input_path(ems, local_path=task_folder, task_id=task_id)
        if Path(ems_path).is_dir():
            ems_path = get_files(ems_path, extensions=[".json"])[0]
        with open(ems_path, "r") as f:
            ems = np.expand_dims(json.load(f)[ems_key], 0)
        try:
            shutil.rmtree(ems_path)
        except:
            os.remove(ems_path)
    else:
        ems = np.expand_dims(np.array(ems), 0)
    return ems


def create_index(dim=768):
    return faiss.IndexFlatL2(dim)


def index_heap(
    df,
    k=1,
    dist_col="distances",
    ids_col="ids",
    size_col="index_size",
    verbose=False,
    with_offset=False,
):
    rh = faiss.ResultHeap(nq=1, k=k)

    ds = df[dist_col].tolist()
    ids = df[ids_col].tolist()
    sz = df[size_col].tolist()
    for i in range(len(ds)):
        if with_offset:
            offset = sum(sz[:i])
        else:
            offset = 0
        msg.info(f"Adding Result: {ds[i]}, {ids[i] + offset}", show=verbose)
        rh.add_result(ds[i], ids[i] + offset)
        msg.good(f"Added Result: {ds[i]}, {ids[i] + offset}", show=verbose)
    rh.finalize()
    return {"distances": rh.D.tolist(), "ids": rh.I.tolist()}


def random_ems(num_ems=100, ems_dim=768, ems_folder="embeddings"):
    "Generate random embeddings and save them to a folder"

    ems = [np.random.random((1, ems_dim))[0].tolist() for i in range(num_ems)]
    os.makedirs(ems_folder, exist_ok=True)
    for i, em in enumerate(ems):
        em_path = str(Path(ems_folder) / f"em_{i}.json")
        json.dump({"embedding": em}, open(em_path, "w"))

In [ ]:
# | hide
# | eval: false


index = create_index(3)
df = pd.DataFrame({"embedding": [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]]})
index.add(np.array([[1, 2, 3]]))
index.search(np.expand_dims(np.array([1, 2, 3]), 0), 1)


(array([[0.]], dtype=float32), array([[0]]))

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()